In [2]:
import numpy as np
import tensorflow as tf

In [3]:
tf.enable_eager_execution()

In [4]:
nbins = 32
width = 1.0/nbins
bins = np.array([[x*width for x in range(nbins+1)] for i in range(3)])

In [5]:
Q = np.random.rand(3,nbins)

In [6]:
total = np.sum(Q,axis=1)

In [7]:
Q[0]=Q[0]/total[0]
Q[1]=Q[1]/total[1]
Q[2]=Q[2]/total[2]

In [8]:
Q = np.insert(Q,0,0,axis=1)

In [80]:
Q

array([[0.        , 0.03930965, 0.00378948, 0.00826786, 0.0135643 ,
        0.02725166, 0.05314366, 0.01577528, 0.06050703, 0.0373432 ,
        0.029346  , 0.03142985, 0.02574737, 0.05562707, 0.05241919,
        0.05938667, 0.04144002, 0.01641293, 0.03455514, 0.00311022,
        0.043208  , 0.05835681, 0.00391468, 0.02628663, 0.03767071,
        0.05577145, 0.01146794, 0.01066007, 0.05492879, 0.00302214,
        0.0342261 , 0.02574026, 0.02631984],
       [0.        , 0.01532583, 0.04751214, 0.01151858, 0.00856559,
        0.01017678, 0.04276045, 0.0392748 , 0.04922023, 0.02108993,
        0.052063  , 0.04882164, 0.05575331, 0.03392851, 0.00719492,
        0.03220565, 0.00017041, 0.04842589, 0.0422421 , 0.03687799,
        0.02392837, 0.01484274, 0.03791074, 0.02552276, 0.03289739,
        0.02827579, 0.05745891, 0.02756339, 0.00537218, 0.01612325,
        0.02008265, 0.05527385, 0.05162021],
       [0.        , 0.03317188, 0.01489491, 0.02381651, 0.04929291,
        0.01297677, 0.0111

In [9]:
xB = np.random.rand(1,3)

In [10]:
xB

array([[0.0223716 , 0.28167491, 0.46267552]])

In [11]:
bins

array([[0.     , 0.03125, 0.0625 , 0.09375, 0.125  , 0.15625, 0.1875 ,
        0.21875, 0.25   , 0.28125, 0.3125 , 0.34375, 0.375  , 0.40625,
        0.4375 , 0.46875, 0.5    , 0.53125, 0.5625 , 0.59375, 0.625  ,
        0.65625, 0.6875 , 0.71875, 0.75   , 0.78125, 0.8125 , 0.84375,
        0.875  , 0.90625, 0.9375 , 0.96875, 1.     ],
       [0.     , 0.03125, 0.0625 , 0.09375, 0.125  , 0.15625, 0.1875 ,
        0.21875, 0.25   , 0.28125, 0.3125 , 0.34375, 0.375  , 0.40625,
        0.4375 , 0.46875, 0.5    , 0.53125, 0.5625 , 0.59375, 0.625  ,
        0.65625, 0.6875 , 0.71875, 0.75   , 0.78125, 0.8125 , 0.84375,
        0.875  , 0.90625, 0.9375 , 0.96875, 1.     ],
       [0.     , 0.03125, 0.0625 , 0.09375, 0.125  , 0.15625, 0.1875 ,
        0.21875, 0.25   , 0.28125, 0.3125 , 0.34375, 0.375  , 0.40625,
        0.4375 , 0.46875, 0.5    , 0.53125, 0.5625 , 0.59375, 0.625  ,
        0.65625, 0.6875 , 0.71875, 0.75   , 0.78125, 0.8125 , 0.84375,
        0.875  , 0.90625, 0.9375 , 0.968

In [18]:
b=tf.searchsorted(bins,tf.transpose(xB),side='right')

In [19]:
tf.transpose(b)

<tf.Tensor: id=106, shape=(1, 3), dtype=int32, numpy=array([[ 1, 10, 15]], dtype=int32)>

In [20]:
Q[:,tf.transpose(b)][0,:]

array([[0.01420116, 0.00373426, 0.00795862]])

In [21]:
Q[:,tf.transpose(b)][:,0]

array([[0.01420116, 0.00373426, 0.00795862],
       [0.00482652, 0.05512942, 0.02111121],
       [0.03036283, 0.05347045, 0.03404066]])

In [22]:
tf.reduce_prod(tf.diag_part(Q[:,tf.transpose(b)][:,0]))

<tf.Tensor: id=123, shape=(), dtype=float64, numpy=2.6650502460893233e-05>

In [24]:
tf.diag_part(bins[:,b[:,0]])

<tf.Tensor: id=136, shape=(3,), dtype=float64, numpy=array([0.03125, 0.3125 , 0.46875])>

In [25]:
b[:,0]-1

<tf.Tensor: id=143, shape=(3,), dtype=int32, numpy=array([ 0,  9, 14], dtype=int32)>

In [27]:
alpha = (xB-tf.diag_part(bins[:,b[:,0]-1]))/width

In [50]:
alpha

<tf.Tensor: id=173, shape=(1, 3), dtype=float64, numpy=array([[0.71589135, 0.01359705, 0.80561679]])>

In [30]:
Q[:,b[:,0]]

array([[0.01420116, 0.00373426, 0.00795862],
       [0.00482652, 0.05512942, 0.02111121],
       [0.03036283, 0.05347045, 0.03404066]])

In [58]:
tf.diag_part(alpha*Q[:,b[:,0]])

<tf.Tensor: id=320, shape=(3,), dtype=float64, numpy=array([0.01016649, 0.0007496 , 0.02742373])>

In [66]:
tf.diag_part(np.cumsum(Q,axis=1)[:,b[:,0]])

<tf.Tensor: id=371, shape=(3,), dtype=float64, numpy=array([0.01420116, 0.38524313, 0.52350583])>

In [69]:
tf.diag_part(alpha*Q[:,b[:,0]])+tf.diag_part(np.cumsum(Q,axis=1)[:,b[:,0]])

<tf.Tensor: id=408, shape=(3,), dtype=float64, numpy=array([0.02436766, 0.38599273, 0.55092956])>